In [1]:
!pip install apache-airflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 

In [4]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import pandas as pd
import logging
from pandas._libs.tslibs.parsing import try_parse_year_month_day
import psycopg2

default_args = {
    'owner': 'MTN Rwanda - Emmanuel Odenyire',
    'depends_on_past': False,
    'start_date': datetime(2023, 4, 23),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5)
}


logging.basicConfig(filename='extract_data.log', level=logging.ERROR)
dag = DAG('data_pipeline', default_args=default_args, schedule ='@daily')

def extract_data():
    try:
        # extracting data from CSV files
        customers = pd.read_csv('customer_data.csv')
        orders = pd.read_csv('order_data.csv')
        payments = pd.read_csv('payment_data.csv')

        # loading the CSV data into Pandas dataframes for later transformation
        return customers, orders, payments
    except Exception as e:
        logging.error(f"An error occurred while extracting data: {e}")

customers, orders, payments = extract_data()

def transform_data(customers, orders, payments):
    # Convert date fields to the correct format using pd.to_datetime
    customers['date_of_birth'] = pd.to_datetime(customers['date_of_birth'])
    orders['order_date'] = pd.to_datetime(orders['order_date'])
    payments['payment_date'] = pd.to_datetime(payments['payment_date'])
    
    # Merge customer and order dataframes on the customer_id column
    customer_orders = pd.merge(customers, orders, on='customer_id')
    
    # Merge payment dataframe with the merged dataframe on the order_id and customer_id columns
    customer_orders_payments = pd.merge(customer_orders, payments, on=['order_id', 'customer_id'])
    
    # Drop unnecessary columns like customer_id and order_id
    customer_orders_payments = customer_orders_payments.drop(['customer_id', 'order_id'], axis=1)
    
    # # Group the data by customer and aggregate the amount paid using sum
    grouped = customer_orders_payments.groupby(['first_name', 'last_name', 'email', 'country', 'gender', 'date_of_birth'])
    #grouped_agg = grouped.agg({'amount': 'sum', 'payment_id': 'nunique', 'order_date': 'max'})
    grouped_agg = grouped.agg({'amount': 'sum', 'payment_id': 'nunique', 'order_date': 'max', 'date_of_birth': 'first'})

    
    # # Create a new column to calculate the total value of orders made by each customer
    grouped_agg['total_order_value'] = grouped_agg['amount'] / grouped_agg['payment_id']
    
    # # Calculate the customer lifetime value using the formula CLV = (average order value) x (number of orders made per year) x (average customer lifespan)
    today = datetime.now()
    grouped_agg['average_customer_lifespan'] = (today - grouped_agg['date_of_birth']).dt.days / 365
    grouped_agg['number_of_orders_per_year'] = grouped_agg['payment_id'] / ((today - grouped_agg['order_date']).dt.days / 365)
    grouped_agg['clv'] = grouped_agg['total_order_value'] * grouped_agg['number_of_orders_per_year'] * grouped_agg['average_customer_lifespan']
  
    #return grouped_agg
    return grouped_agg

def load_data(grouped_agg):
    # Connect to the Postgres database

    """
    POSTGRES_ADDRESS = '35.237.226.12'
    POSTGRES_PORT = '5432'
    POSTGRES_USERNAME = 'postgres'
    POSTGRES_PASSWORD = 'password'
    POSTGRES_DBNAME = 'telecommunications_data'
    """
    
    try:
        conn = psycopg2.connect(database="telecommunications_data", user="postgres", password="password", host="35.237.226.12", port="5432")
    except psycopg2.Error as e:
        logging.error("Unable to connect to the database: %s", e)
        return

    # Create the necessary tables if it does not already exist
    with conn.cursor() as cur:
        try:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS payments_customers_orders (
                    first_name VARCHAR(50),
                    last_name VARCHAR(50),
                    email VARCHAR(255),
                    country VARCHAR(50),
                    gender VARCHAR(10),
                    date_of_birth DATE,
                    amount INTEGER,
                    payment_id INTEGER,
                    order_date DATE,
                    total_order_value INTEGER,
                    average_customer_lifespan FLOAT,
                    number_of_orders_per_year FLOAT,
                    clv FLOAT
                )
            """)
            conn.commit()
        except psycopg2.Error as e:
            logging.error("Error creating table: %s", e)
            return

    # Loop through the dataframe and insert the data into the table using SQL INSERT statements
    with conn.cursor() as cur:
        try:
            for row in grouped_agg.itertuples():
                cur.execute("""
                    INSERT INTO payments_customers_orders (
                        first_name,
                        last_name,
                        email,
                        country,
                        gender,
                        date_of_birth,
                        amount,
                        payment_id,
                        order_date,
                        total_order_value,
                        average_customer_lifespan,
                        number_of_orders_per_year,
                        clv
                    ) VALUES (
                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                    )
                """, row[1:])
            conn.commit()
        except psycopg2.Error as e:
            logging.error("Error inserting data into table: %s", e)
            return

    # commit the changes and close the connection
    cur.close()
    conn.close()

# define the tasks
extract_task = PythonOperator(
    task_id='extract_data',
    python_callable=extract_data,
    dag=dag
)

transform_task = PythonOperator(
    task_id='transform_data',
    python_callable=transform_data,
    dag=dag
)

load_task = PythonOperator(
    task_id='load_data',
    python_callable=load_data,
    dag=dag
)

# define the dependencies
extract_task >> transform_task >> load_task



<ipython-input-4-4125ad921393>:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

<Task(PythonOperator): load_data>